In [4]:
from sklearn.neural_network import MLPClassifier
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE  
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
import math
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')

In [ ]:


patient_all = []

for i in range(31):
    #path = "/Users/Chloe/City, University of London/PG-Lopez, Damien - INM701/MHCD/motion"
    #path = "/Users/damie/OneDrive - City, University of London/INM701/MHCD/motion"
    path = "/Users/willemvandemierop/Documents/Master AI/Introduction to AI/Coursework/MHCD/motion"
    filename_read = os.path.join(path, f"A{i}.txt")
    patient_temp_accel = pd.read_csv(filename_read, sep=" ", header=None, names=["time", "x", "y", "z"])
    # remove negative time
    # patient_temp_accel = patient_temp_accel.drop(patient_temp_accel[patient_temp_accel.time < 0].index)
    # round time to nearest whole number
    patient_temp_accel['time'] = patient_temp_accel['time'].round(decimals=-1)
    # only every 30s
    # patient_temp_accel = patient_temp_accel[::1500]
    patient_temp_accel = patient_temp_accel.loc[patient_temp_accel['time'] % 30 == 0]
    # remove duplicates, keep first
    patient_temp_accel = patient_temp_accel.drop_duplicates(subset='time', keep='first', inplace=False)

    #path = "/Users/Chloe/City, University of London/PG-Lopez, Damien - INM701/MHCD/heart_rate"
    #path = "/Users/damie/OneDrive - City, University of London/INM701/MHCD/heart_rate"
    path = "/Users/willemvandemierop/Documents/Master AI/Introduction to AI/Coursework/MHCD/heart_rate"
    filename_read = os.path.join(path, f"H{i}.txt")
    patient_temp_heartbeat = pd.read_csv(filename_read, sep=",", header=None, names=["time", "heartbeat"])
    patient_temp_heartbeat['time'] = patient_temp_heartbeat['time'].round(decimals=-1)
    # patient_temp_heartbeat = patient_temp_heartbeat.drop(patient_temp_heartbeat[patient_temp_heartbeat.time < 0].index)
    patient_temp_heartbeat = patient_temp_heartbeat.loc[patient_temp_heartbeat['time'] % 30 == 0]
    patient_temp_heartbeat = patient_temp_heartbeat.drop_duplicates(subset='time', keep='first', inplace=False)

    #path = "/Users/Chloe/City, University of London/PG-Lopez, Damien - INM701/MHCD/labels"
    #path = "/Users/damie/OneDrive - City, University of London/INM701/MHCD/labels"
    path =  "/Users/willemvandemierop/Documents/Master AI/Introduction to AI/Coursework/MHCD/labels"
    filename_read = os.path.join(path, f"L{i}.txt")
    patient_temp_label = pd.read_csv(filename_read, sep=" ", header=None, names=["time", "label"])

    # merge dataframes
    patient_temp_all = pd.merge(pd.merge(patient_temp_accel, patient_temp_heartbeat, on='time'), patient_temp_label,
                                on='time')
    #print(patient_temp_all)
    patient_all.append(patient_temp_all)

print("\n Data correctly loaded\n ")

In [ ]:
PAL = pd.concat(patient_all)
print("All patients\n", PAL.head())
export_csv = PAL.to_csv (r'/Users/willemvandemierop/Documents/Master AI/Introduction to AI/Coursework/All_patients_correct2.csv', index = None, header=True)

In [5]:
#for section 'Adding an extra feature to the classifier'

def circadian(time):
    return np.cos((math.pi*2/24)*(time/60/60))

patient_all_circadian = []

for i in range(31):
    path = "/Users/Chloe/City, University of London/PG-Lopez, Damien - INM701/MHCD/motion"
    #path = "/Users/damie/OneDrive - City, University of London/INM701/MHCD/motion"
    #path = "/Users/willemvandemierop/Documents/Master AI/Introduction to AI/Coursework/MHCD/motion"
    filename_read = os.path.join(path, f"A{i}.txt")
    patient_temp_accel = pd.read_csv(filename_read, sep=" ", header=None, names=["time", "x", "y", "z"])
    # remove negative time
    patient_temp_accel = patient_temp_accel.drop(patient_temp_accel[patient_temp_accel.time < 0].index)
    # round time to nearest whole number
    patient_temp_accel['time'] = patient_temp_accel['time'].round(decimals=-1)
    # only every 30s
    patient_temp_accel = patient_temp_accel.loc[patient_temp_accel['time'] % 30 == 0]
    # remove duplicates, keep first
    patient_temp_accel = patient_temp_accel.drop_duplicates(subset='time', keep='first', inplace=False)

    path = "/Users/Chloe/City, University of London/PG-Lopez, Damien - INM701/MHCD/heart_rate"
    #path = "/Users/damie/OneDrive - City, University of London/INM701/MHCD/heart_rate"
    #path = "/Users/willemvandemierop/Documents/Master AI/Introduction to AI/Coursework/MHCD/heart_rate"
    filename_read = os.path.join(path, f"H{i}.txt")
    patient_temp_heartbeat = pd.read_csv(filename_read, sep=",", header=None, names=["time", "heartbeat"])
    patient_temp_heartbeat['time'] = patient_temp_heartbeat['time'].round(decimals=-1)
    patient_temp_heartbeat = patient_temp_heartbeat.drop(patient_temp_heartbeat[patient_temp_heartbeat.time < 0].index)
    patient_temp_heartbeat = patient_temp_heartbeat.loc[patient_temp_heartbeat['time'] % 30 == 0]
    patient_temp_heartbeat = patient_temp_heartbeat.drop_duplicates(subset='time', keep='first', inplace=False)

    path = "/Users/Chloe/City, University of London/PG-Lopez, Damien - INM701/MHCD/labels"
    #path = "/Users/damie/OneDrive - City, University of London/INM701/MHCD/labels"
    #path =  "/Users/willemvandemierop/Documents/Master AI/Introduction to AI/Coursework/MHCD/labels"
    filename_read = os.path.join(path, f"L{i}.txt")
    patient_temp_label = pd.read_csv(filename_read, sep=" ", header=None, names=["time", "label"])

    # merge dataframes
    patient_temp_all = pd.merge(pd.merge(patient_temp_accel, patient_temp_heartbeat, on='time'), patient_temp_label,
                                on='time')
    
    #add cos func of time 
    patient_temp_all.insert(6,'circadian', circadian(patient_temp_all['time']))
    
    #print(patient_temp_all)
    patient_all_circadian.append(patient_temp_all)

print("\n Data correctly loaded\n ")


 Data correctly loaded
 


In [ ]:
PAL = pd.concat(patient_all_circadian)
print("All patients\n", PAL.head())
export_csv = PAL.to_csv (r'/Users/Chloe/Desktop/AI MSc/Intro to AI/Coursework/All_patients_correct_circadian.csv', index = None, header=True)
